In [28]:
import json
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer 
import requests # Vẫn có thể cần cho các thao tác API khác nếu không qua LangChain hết

# LangChain components
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain

# Tải các biến môi trường từ file .env
load_dotenv()

# Neo4j Config
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "Muclytamdaithien"

# ChatGroq Config (Llama 3)
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
# GROQ_API_URL không cần thiết nữa khi dùng LangChain wrapper

# Embedding Model (vẫn có thể cần nếu bạn muốn RAG trên các node luật chi tiết)
EMBEDDING_MODEL_NAME = 'bkai-foundation-models/vietnamese-bi-encoder'

print("Thư viện và cấu hình đã được tải (bao gồm LangChain).")
if not GROQ_API_KEY:
    print("LỖI: GROQ_API_KEY chưa được thiết lập trong file .env!")

Thư viện và cấu hình đã được tải (bao gồm LangChain).


In [29]:
driver = None
llm = None
embedding_model = None # Vẫn giữ nếu có kế hoạch dùng

# Kết nối Neo4j
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
driver.verify_connectivity()
print("Successfully connected to Neo4j.")

# Khởi tạo LLM với LangChain (ChatGroq)
if GROQ_API_KEY:
    llm = ChatGroq(
        groq_api_key=GROQ_API_KEY,
        model_name="llama3-8b-8192", # Hoặc model bạn muốn dùng: "mixtral-8x7b-32768"
        temperature=0.5 # Giảm nhiệt độ để bám sát template
    )
    print("ChatGroq LLM (Llama 3) initialized via LangChain.")
else:
    print("Không thể khởi tạo LLM do thiếu GROQ_API_KEY.")

# Tải embedding model (nếu vẫn cần cho RAG chi tiết)
# print(f"Loading embedding model: {EMBEDDING_MODEL_NAME}...")
# embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME)
# print("Embedding model loaded.")

Successfully connected to Neo4j.
ChatGroq LLM (Llama 3) initialized via LangChain.


In [31]:
def clear_database(tx):
    tx.run("MATCH (n) DETACH DELETE n")
    print("Database cleared.")

def create_constraints(tx):
    tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (ld:LawDocument) REQUIRE ld.law_document_id IS UNIQUE")
    tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (art:Article) REQUIRE art.article_id IS UNIQUE")
    tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (cls:Clause) REQUIRE cls.clause_id IS UNIQUE")
    tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (pnt:Point) REQUIRE pnt.point_id IS UNIQUE")
    print("Constraints created or already exist.")

#Chạy các hàm này nếu bạn muốn làm mới KG
with driver.session(database="neo4j") as session:
    session.execute_write(clear_database)
    session.execute_write(create_constraints)
print("Database preparation complete (if uncommented).")

Database cleared.
Constraints created or already exist.
Database preparation complete (if uncommented).


In [32]:
law_data = {
    "LHNGD2014": {
        "law_document_id": "LHNGD2014",
        "law_document_name": "Luật Hôn nhân và gia đình năm 2014",
        "articles": {
            "ARTICLE_LHNGD2014_DIEU5": {
                "article_number_text": "Điều 5",
                "article_name": "Bảo vệ chế độ hôn nhân và gia đình",
                "article_content": "1. Quan hệ hôn nhân và gia đình được xác lập, thực hiện theo quy định của Luật này được tôn trọng và được pháp luật bảo vệ.\n2. Cấm các hành vi sau đây:\na) Kết hôn giả tạo, ly hôn giả tạo;\nb) Tảo hôn, cưỡng ép kết hôn, lừa dối kết hôn, cản trở kết hôn;\nc) Người đang có vợ, có chồng mà kết hôn hoặc chung sống như vợ chồng với người khác hoặc chưa có vợ, chưa có chồng mà kết hôn hoặc chung sống như vợ chồng với người đang có chồng, có vợ;\nd) Kết hôn hoặc chung sống như vợ chồng giữa những người cùng dòng máu về trực hệ; giữa những người có họ trong phạm vi ba đời; giữa cha, mẹ nuôi với con nuôi; giữa người đã từng là cha, mẹ nuôi với con nuôi, cha chồng với con dâu, mẹ vợ với con rể, cha dượng với con riêng của vợ, mẹ kế với con riêng của chồng;\nđ) Yêu sách của cải trong kết hôn;\ne) Cưỡng ép ly hôn, lừa dối ly hôn, cản trở ly hôn;\ng) Thực hiện sinh con bằng kỹ thuật hỗ trợ sinh sản vì mục đích thương mại, mang thai hộ vì mục đích thương mại, lựa chọn giới tính thai nhi, sinh sản vô tính;\nh) Bạo lực gia đình;\ni) Lợi dụng việc thực hiện quyền về hôn nhân và gia đình để mua bán người, bóc lột sức lao động, xâm phạm tình dục hoặc có hành vi khác nhằm mục đích trục lợi.\n3. Mọi hành vi vi phạm pháp luật về hôn nhân và gia đình phải được xử lý nghiêm minh, đúng pháp luật.\n4. Cơ quan, tổ chức, cá nhân có quyền yêu cầu Tòa án, cơ quan khác có thẩm quyền áp dụng biện pháp kịp thời ngăn chặn và xử lý người có hành vi vi phạm pháp luật về hôn nhân và gia đình.",
                "clauses": {
                    "CLAUSE_LHNGD2014_DIEU5_KHOAN2": {
                        "clause_number_text": "Khoản 2",
                        "clause_content": "Cấm các hành vi sau đây:\na) Kết hôn giả tạo, ly hôn giả tạo;\nb) Tảo hôn, cưỡng ép kết hôn, lừa dối kết hôn, cản trở kết hôn;\nc) Người đang có vợ, có chồng mà kết hôn hoặc chung sống như vợ chồng với người khác hoặc chưa có vợ, chưa có chồng mà kết hôn hoặc chung sống như vợ chồng với người đang có chồng, có vợ;\nd) Kết hôn hoặc chung sống như vợ chồng giữa những người cùng dòng máu về trực hệ; giữa những người có họ trong phạm vi ba đời; giữa cha, mẹ nuôi với con nuôi; giữa người đã từng là cha, mẹ nuôi với con nuôi, cha chồng với con dâu, mẹ vợ với con rể, cha dượng với con riêng của vợ, mẹ kế với con riêng của chồng;\nđ) Yêu sách của cải trong kết hôn;\ne) Cưỡng ép ly hôn, lừa dối ly hôn, cản trở ly hôn;\ng) Thực hiện sinh con bằng kỹ thuật hỗ trợ sinh sản vì mục đích thương mại, mang thai hộ vì mục đích thương mại, lựa chọn giới tính thai nhi, sinh sản vô tính;\nh) Bạo lực gia đình;\ni) Lợi dụng việc thực hiện quyền về hôn nhân và gia đình để mua bán người, bóc lột sức lao động, xâm phạm tình dục hoặc có hành vi khác nhằm mục đích trục lợi.",
                        "points": {
                            "POINT_LHNGD2014_DIEU5_KHOAN2_DIEMA": {"point_letter_text": "a", "point_content": "Kết hôn giả tạo, ly hôn giả tạo;"},
                            "POINT_LHNGD2014_DIEU5_KHOAN2_DIEMB": {"point_letter_text": "b", "point_content": "Tảo hôn, cưỡng ép kết hôn, lừa dối kết hôn, cản trở kết hôn;"},
                            "POINT_LHNGD2014_DIEU5_KHOAN2_DIEMC": {"point_letter_text": "c", "point_content": "Người đang có vợ, có chồng mà kết hôn hoặc chung sống như vợ chồng với người khác hoặc chưa có vợ, chưa có chồng mà kết hôn hoặc chung sống như vợ chồng với người đang có chồng, có vợ;"},
                            "POINT_LHNGD2014_DIEU5_KHOAN2_DIEMD": {"point_letter_text": "d", "point_content": "Kết hôn hoặc chung sống như vợ chồng giữa những người cùng dòng máu về trực hệ; giữa những người có họ trong phạm vi ba đời; giữa cha, mẹ nuôi với con nuôi; giữa người đã từng là cha, mẹ nuôi với con nuôi, cha chồng với con dâu, mẹ vợ với con rể, cha dượng với con riêng của vợ, mẹ kế với con riêng của chồng;"},
                            "POINT_LHNGD2014_DIEU5_KHOAN2_DIEMDD": {"point_letter_text": "đ", "point_content": "Yêu sách của cải trong kết hôn;"},
                            "POINT_LHNGD2014_DIEU5_KHOAN2_DIEME": {"point_letter_text": "e", "point_content": "Cưỡng ép ly hôn, lừa dối ly hôn, cản trở ly hôn;"},
                            "POINT_LHNGD2014_DIEU5_KHOAN2_DIEMG": {"point_letter_text": "g", "point_content": "Thực hiện sinh con bằng kỹ thuật hỗ trợ sinh sản vì mục đích thương mại, mang thai hộ vì mục đích thương mại, lựa chọn giới tính thai nhi, sinh sản vô tính;"},
                            "POINT_LHNGD2014_DIEU5_KHOAN2_DIEMH": {"point_letter_text": "h", "point_content": "Bạo lực gia đình;"},
                            "POINT_LHNGD2014_DIEU5_KHOAN2_DIEMI": {"point_letter_text": "i", "point_content": "Lợi dụng việc thực hiện quyền về hôn nhân và gia đình để mua bán người, bóc lột sức lao động, xâm phạm tình dục hoặc có hành vi khác nhằm mục đích trục lợi."}
                        }
                    }
                }
            },
            "ARTICLE_LHNGD2014_DIEU8": {
                "article_number_text": "Điều 8",
                "article_name": "Điều kiện kết hôn",
                "article_content": "1. Nam, nữ kết hôn với nhau phải tuân theo các điều kiện sau đây:\na) Nam từ đủ 20 tuổi trở lên, nữ từ đủ 18 tuổi trở lên;\nb) Việc kết hôn do nam và nữ tự nguyện quyết định;\nc) Không bị mất năng lực hành vi dân sự;\nd) Việc kết hôn không thuộc một trong các trường hợp cấm kết hôn theo quy định tại các điểm a, b, c và d khoản 2 Điều 5 của Luật này.\n2. Nhà nước không thừa nhận hôn nhân giữa những người cùng giới tính.",
                "clauses": {
                    "CLAUSE_LHNGD2014_DIEU8_KHOAN1": {
                        "clause_number_text": "Khoản 1",
                        "clause_content": "Nam, nữ kết hôn với nhau phải tuân theo các điều kiện sau đây:\na) Nam từ đủ 20 tuổi trở lên, nữ từ đủ 18 tuổi trở lên;\nb) Việc kết hôn do nam và nữ tự nguyện quyết định;\nc) Không bị mất năng lực hành vi dân sự;\nd) Việc kết hôn không thuộc một trong các trường hợp cấm kết hôn theo quy định tại các điểm a, b, c và d khoản 2 Điều 5 của Luật này.",
                        "points": {
                            "POINT_LHNGD2014_DIEU8_KHOAN1_DIEMA": {"point_letter_text": "a", "point_content": "Nam từ đủ 20 tuổi trở lên, nữ từ đủ 18 tuổi trở lên;"},
                            "POINT_LHNGD2014_DIEU8_KHOAN1_DIEMB": {"point_letter_text": "b", "point_content": "Việc kết hôn do nam và nữ tự nguyện quyết định;"},
                            "POINT_LHNGD2014_DIEU8_KHOAN1_DIEMC": {"point_letter_text": "c", "point_content": "Không bị mất năng lực hành vi dân sự;"},
                            "POINT_LHNGD2014_DIEU8_KHOAN1_DIEMD": {"point_letter_text": "d", "point_content": "Việc kết hôn không thuộc một trong các trường hợp cấm kết hôn theo quy định tại các điểm a, b, c và d khoản 2 Điều 5 của Luật này."}
                        }
                    },
                     "CLAUSE_LHNGD2014_DIEU8_KHOAN2": {
                        "clause_number_text": "Khoản 2",
                        "clause_content": "Nhà nước không thừa nhận hôn nhân giữa những người cùng giới tính."
                    }
                }
            }
        }
    }
}
print("Dữ liệu luật mẫu đã được định nghĩa.")

Dữ liệu luật mẫu đã được định nghĩa.


In [33]:
def create_law_kg(tx, law_data_dict):
    for law_doc_id_key, law_doc_details in law_data_dict.items():
        tx.run("""
            MERGE (ld:LawDocument {law_document_id: $id})
            SET ld.name = $name
        """, id=law_doc_details["law_document_id"], name=law_doc_details["law_document_name"])
        print(f"Created/Updated LawDocument: {law_doc_details['law_document_id']}")

        for article_id_key, article_details in law_doc_details.get("articles", {}).items():
            # article_embedding = embedding_model.encode(article_details["article_content"]).tolist() # Tạm bỏ embedding ở đây
            tx.run("""
                MERGE (art:Article {article_id: $id})
                SET art.number_text = $num_text,
                    art.name = $name,
                    art.content = $content
                MERGE (ld:LawDocument {law_document_id: $law_doc_id})
                MERGE (art)-[:IS_PART_OF_DOCUMENT]->(ld)
            """, id=article_id_key, num_text=article_details["article_number_text"],
                 name=article_details.get("article_name", ""), content=article_details["article_content"],
                 # embedding=article_embedding, # Bỏ embedding
                 law_doc_id=law_doc_details["law_document_id"])
            print(f"  Created/Updated Article: {article_id_key}")

            for clause_id_key, clause_details in article_details.get("clauses", {}).items():
                # clause_embedding = embedding_model.encode(clause_details["clause_content"]).tolist() # Tạm bỏ embedding
                tx.run("""
                    MERGE (cls:Clause {clause_id: $id})
                    SET cls.number_text = $num_text,
                        cls.content = $content
                    MERGE (art:Article {article_id: $article_id})
                    MERGE (cls)-[:IS_PART_OF_ARTICLE]->(art)
                """, id=clause_id_key, num_text=clause_details["clause_number_text"],
                     content=clause_details["clause_content"], # embedding=clause_embedding, # Bỏ embedding
                     article_id=article_id_key)
                print(f"    Created/Updated Clause: {clause_id_key}")

                for point_id_key, point_details in clause_details.get("points", {}).items():
                    # point_embedding = embedding_model.encode(point_details["point_content"]).tolist() # Tạm bỏ embedding
                    tx.run("""
                        MERGE (pnt:Point {point_id: $id})
                        SET pnt.letter_text = $letter_text,
                            pnt.content = $content
                        MERGE (cls:Clause {clause_id: $clause_id})
                        MERGE (pnt)-[:IS_PART_OF_CLAUSE]->(cls)
                    """, id=point_id_key, letter_text=point_details["point_letter_text"],
                         content=point_details["point_content"], # embedding=point_embedding, # Bỏ embedding
                         clause_id=clause_id_key)
                    print(f"      Created/Updated Point: {point_id_key}")

#Chạy hàm để tạo KG (chỉ một lần hoặc khi dữ liệu luật thay đổi)
with driver.session(database="neo4j") as session:
   session.execute_write(create_law_kg, law_data)
print("Knowledge Graph for Law data created/updated.")

Created/Updated LawDocument: LHNGD2014
  Created/Updated Article: ARTICLE_LHNGD2014_DIEU5
    Created/Updated Clause: CLAUSE_LHNGD2014_DIEU5_KHOAN2
      Created/Updated Point: POINT_LHNGD2014_DIEU5_KHOAN2_DIEMA
      Created/Updated Point: POINT_LHNGD2014_DIEU5_KHOAN2_DIEMB
      Created/Updated Point: POINT_LHNGD2014_DIEU5_KHOAN2_DIEMC
      Created/Updated Point: POINT_LHNGD2014_DIEU5_KHOAN2_DIEMD
      Created/Updated Point: POINT_LHNGD2014_DIEU5_KHOAN2_DIEMDD
      Created/Updated Point: POINT_LHNGD2014_DIEU5_KHOAN2_DIEME
      Created/Updated Point: POINT_LHNGD2014_DIEU5_KHOAN2_DIEMG
      Created/Updated Point: POINT_LHNGD2014_DIEU5_KHOAN2_DIEMH
      Created/Updated Point: POINT_LHNGD2014_DIEU5_KHOAN2_DIEMI
  Created/Updated Article: ARTICLE_LHNGD2014_DIEU8
    Created/Updated Clause: CLAUSE_LHNGD2014_DIEU8_KHOAN1
      Created/Updated Point: POINT_LHNGD2014_DIEU8_KHOAN1_DIEMA
      Created/Updated Point: POINT_LHNGD2014_DIEU8_KHOAN1_DIEMB
      Created/Updated Point: POINT_LHN

In [34]:
def get_law_content_from_kg(tx, article_id_list, clause_id_list=None, point_id_list=None):
    contents = {}
    for article_id in article_id_list:
        result = tx.run("MATCH (a:Article {article_id: $id}) RETURN a.content AS content, a.number_text AS number, a.name AS name", id=article_id)
        record = result.single()
        if record:
            contents[article_id] = {
                "type": "Article",
                "number": record["number"],
                "name": record["name"],
                "content": record["content"]
            }
    if clause_id_list:
        for clause_id in clause_id_list:
            result = tx.run("""
                MATCH (c:Clause {clause_id: $id})-[:IS_PART_OF_ARTICLE]->(a:Article)
                RETURN c.content AS content, c.number_text AS number, a.number_text AS article_number
            """, id=clause_id)
            record = result.single()
            if record:
                contents[clause_id] = {
                    "type": "Clause",
                    "number": record["number"],
                    "article_number": record["article_number"],
                    "content": record["content"]
                }
    if point_id_list:
        for point_id in point_id_list:
            result = tx.run("""
                MATCH (p:Point {point_id: $id})-[:IS_PART_OF_CLAUSE]->(c:Clause)-[:IS_PART_OF_ARTICLE]->(a:Article)
                RETURN p.content AS content, p.letter_text AS letter, c.number_text AS clause_number, a.number_text AS article_number
            """, id=point_id)
            record = result.single()
            if record:
                contents[point_id] = {
                    "type": "Point",
                    "letter": record["letter"],
                    "clause_number": record["clause_number"],
                    "article_number": record["article_number"],
                    "content": record["content"]
                }
    return contents

print("Hàm truy xuất nội dung luật từ KG đã sẵn sàng.")

Hàm truy xuất nội dung luật từ KG đã sẵn sàng.


In [35]:
# Định nghĩa few-shot example của bạn
FEW_SHOT_QUESTION = "Có được đăng ký kết hôn trong phạm vi ba đời không?"
FEW_SHOT_ANSWER = """Căn cứ theo Điều 8 Luật Hôn nhân và gia đình 2014 quy định về điều kiện kết hôn như sau:
Điều 8. Điều kiện kết hôn
1. Nam, nữ kết hôn với nhau phải tuân theo các điều kiện sau đây:
a) Nam từ đủ 20 tuổi trở lên, nữ từ đủ 18 tuổi trở lên;
b) Việc kết hôn do nam và nữ tự nguyện quyết định;
c) Không bị mất năng lực hành vi dân sự;
d) Việc kết hôn không thuộc một trong các trường hợp cấm kết hôn theo quy định tại các điểm a, b, c và d khoản 2 Điều 5 của Luật này.
2. Nhà nước không thừa nhận hôn nhân giữa những người cùng giới tính.

Căn cứ theo Điều 5 Luật Hôn nhân và gia đình 2014 quy định về bảo vệ chế độ hôn nhân và gia đình như sau:
Điều 5. Bảo vệ chế độ hôn nhân và gia đình
1. Quan hệ hôn nhân và gia đình được xác lập, thực hiện theo quy định của Luật này được tôn trọng và được pháp luật bảo vệ.
2. Cấm các hành vi sau đây:
a) Kết hôn giả tạo, ly hôn giả tạo;
b) Tảo hôn, cưỡng ép kết hôn, lừa dối kết hôn, cản trở kết hôn;
c) Người đang có vợ, có chồng mà kết hôn hoặc chung sống như vợ chồng với người khác hoặc chưa có vợ, chưa có chồng mà kết hôn hoặc chung sống như vợ chồng với người đang có chồng, có vợ;
d) Kết hôn hoặc chung sống như vợ chồng giữa những người cùng dòng máu về trực hệ; giữa những người có họ trong phạm vi ba đời; giữa cha, mẹ nuôi với con nuôi; giữa người đã từng là cha, mẹ nuôi với con nuôi, cha chồng với con dâu, mẹ vợ với con rể, cha dượng với con riêng của vợ, mẹ kế với con riêng của chồng;
đ) Yêu sách của cải trong kết hôn;
e) Cưỡng ép ly hôn, lừa dối ly hôn, cản trở ly hôn;
g) Thực hiện sinh con bằng kỹ thuật hỗ trợ sinh sản vì mục đích thương mại, mang thai hộ vì mục đích thương mại, lựa chọn giới tính thai nhi, sinh sản vô tính;
h) Bạo lực gia đình;
i) Lợi dụng việc thực hiện quyền về hôn nhân và gia đình để mua bán người, bóc lột sức lao động, xâm phạm tình dục hoặc có hành vi khác nhằm mục đích trục lợi.
3. Mọi hành vi vi phạm pháp luật về hôn nhân và gia đình phải được xử lý nghiêm minh, đúng pháp luật.
4. Cơ quan, tổ chức, cá nhân có quyền yêu cầu Tòa án, cơ quan khác có thẩm quyền áp dụng biện pháp kịp thời ngăn chặn và xử lý người có hành vi vi phạm pháp luật về hôn nhân và gia đình.

Theo đó, kết hôn trong phạm vi ba đời thuộc một trong những hành vi bị pháp luật cấm trong bảo vệ chế độ hôn nhân và gia đình, cụ thể tại Điểm d Khoản 2 Điều 5.
Như vậy, không được đăng ký kết hôn trong phạm vi ba đời.
"""

# Tạo danh sách các ví dụ cho FewShotPromptTemplate
examples = [
    {
        "user_query": FEW_SHOT_QUESTION,
        "dieu8_content": law_data["LHNGD2014"]["articles"]["ARTICLE_LHNGD2014_DIEU8"]["article_content"],
        "dieu5_content": law_data["LHNGD2014"]["articles"]["ARTICLE_LHNGD2014_DIEU5"]["article_content"],
        "specific_point_info": f"Lưu ý cụ thể đến hành vi cấm tại Điểm d Khoản 2 Điều 5: '{law_data['LHNGD2014']['articles']['ARTICLE_LHNGD2014_DIEU5']['clauses']['CLAUSE_LHNGD2014_DIEU5_KHOAN2']['points']['POINT_LHNGD2014_DIEU5_KHOAN2_DIEMD']['point_content']}'", # Ví dụ
        "answer": FEW_SHOT_ANSWER
    }
]

# Tạo một template cho ví dụ
example_prompt_template = PromptTemplate(
    input_variables=["user_query", "dieu8_content", "dieu5_content", "specific_point_info", "answer"],
    template="Câu hỏi ví dụ: {user_query}\nTrả lời ví dụ:\nCăn cứ theo Điều 8 Luật Hôn nhân và gia đình 2014 quy định về điều kiện kết hôn như sau:\n{dieu8_content}\n\nCăn cứ theo Điều 5 Luật Hôn nhân và gia đình 2014 quy định về bảo vệ chế độ hôn nhân và gia đình như sau:\n{dieu5_content}\n\n{specific_point_info}\n\n{answer}"
)

# Tạo FewShotPromptTemplate
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt_template,
    prefix="Bạn là một trợ lý pháp lý am hiểu Luật Hôn nhân và Gia đình Việt Nam. Hãy trả lời câu hỏi của người dùng một cách chi tiết, trích dẫn đầy đủ điều luật liên quan và đưa ra kết luận rõ ràng theo phong cách của ví dụ đã cho.\nĐây là một ví dụ về cách trả lời câu hỏi pháp lý về điều kiện kết hôn:",
    suffix="Bây giờ, hãy trả lời câu hỏi sau đây dựa trên cùng một phong cách và dựa vào các điều luật được cung cấp.\nLuật tham khảo:\nĐiều 8 Luật Hôn nhân và gia đình 2014:\n{dieu8_content}\n\nĐiều 5 Luật Hôn nhân và gia đình 2014 (đặc biệt lưu ý Khoản 2 về các hành vi cấm):\n{dieu5_content}\n{specific_point_info}\n\nCâu hỏi người dùng: {user_query}\nTrả lời:",
    input_variables=["user_query", "dieu8_content", "dieu5_content", "specific_point_info"] # Các biến sẽ được truyền vào khi chạy
)

# Tạo một chain LLM
if llm:
    law_qa_chain = LLMChain(llm=llm, prompt=few_shot_prompt, output_parser=StrOutputParser())
    print("LangChain FewShotPromptTemplate và LLMChain đã được khởi tạo.")
else:
    law_qa_chain = None
    print("LLMChain không được khởi tạo do LLM chưa sẵn sàng.")

LangChain FewShotPromptTemplate và LLMChain đã được khởi tạo.


In [36]:
prompt_template = """
Bạn là một trợ lý pháp lý am hiểu Luật Hôn nhân và Gia đình Việt Nam.
Hãy trả lời câu hỏi của người dùng một cách chi tiết, tuân theo cấu trúc sau:
1. Trích dẫn và phân tích các điều kiện kết hôn tại Điều 8 Luật HNGĐ 2014.
2. Nếu câu hỏi liên quan đến một trường hợp có thể bị cấm (như được đề cập tại điểm d khoản 1 Điều 8), hãy trích dẫn và phân tích các hành vi bị cấm liên quan tại Khoản 2 Điều 5 Luật HNGĐ 2014.
3. Đưa ra kết luận cuối cùng cho câu hỏi.

Dưới đây là một ví dụ về cách đưa ra câu trả lời.
Câu hỏi ví dụ: Việt Nam có chấp nhận hôn nhân đồng giới không?
Trả lời ví dụ:
Căn cứ theo quy định tại Điều 8 Luật Hôn nhân và gia đình 2014 về điều kiện kết hôn như sau:
Điều kiện kết hôn
1. Nam, nữ kết hôn với nhau phải tuân theo các điều kiện sau đây:
a) Nam từ đủ 20 tuổi trở lên, nữ từ đủ 18 tuổi trở lên;
b) Việc kết hôn do nam và nữ tự nguyện quyết định;
c) Không bị mất năng lực hành vi dân sự;
d) Việc kết hôn không thuộc một trong các trường hợp cấm kết hôn theo quy định tại các điểm a, b, c và d khoản 2 Điều 5 của Luật này.
2. Nhà nước không thừa nhận hôn nhân giữa những người cùng giới tính.

Theo đó tại Luật Hôn nhân và gia đình 2014 đang có hiệu lực thi hành điều chỉnh những vẫn đề liên quan đến chế độ hôn nhân và gia đình hiện nay đã nêu rõ rằng Nhà nước không thừa nhận hôn nhân giữa những người cùng giới tính tức là không thừa nhận hôn nhân đồng giới
Tuy nhiên theo quy định cũng đã nêu rõ, Nhà nước không thừa nhận hôn nhân đồng giới chứ không cấm hôn nhân đồng giới
Tức là các cặp vợ chồng đồng giới đang sinh sống tại Việt Nam và đã kết hôn hợp pháp theo quy định của pháp luật nước ngoài thì cũng không phải là vi phạm pháp luật Việt Nam

---
Bây giờ, hãy trả lời câu hỏi sau:

Ngữ cảnh pháp lý:
Điều 8 Luật Hôn nhân và Gia đình 2014:
{dieu8_content_from_kg}

Khoản 2 Điều 5 Luật Hôn nhân và Gia đình 2014:
{khoan2_dieu5_content_from_kg}

Những chi tiết liên quan ở điều 8: (Sử dụng để làm rõ phân tích của bạn)
{dieu8_content_from_rag}

Những chi tiết có thể liên quan ở Điều 5: (Có thể (sử dụng hoặc không) để làm rõ phân tích của bạn)
{dieu5_content_from_rag}

Câu hỏi người dùng: {user_query}
Trả lời:
"""

In [38]:
user_question_1 = "Chưa được xóa án tích có được đăng ký kết hôn không?"

questions_to_test = [user_question_1]

if driver and law_qa_chain:
    # --- SỬA ĐỔI QUAN TRỌNG: Lấy nội dung luật từ KG một cách chính xác ---
    # Chúng ta sẽ lấy nội dung này MỘT LẦN và truyền vào chain cho mỗi câu hỏi
    dieu8_full_content = "N/A"
    dieu5_full_content = "N/A"
    specific_points_data = {} # Để lưu trữ các điểm cụ thể nếu cần

    with driver.session(database="neo4j") as session:
        # Lấy nội dung Điều 8
        result_dieu8 = session.run("MATCH (a:Article {article_id: 'ARTICLE_LHNGD2014_DIEU8'}) RETURN a.content AS content")
        record_dieu8 = result_dieu8.single()
        if record_dieu8:
            dieu8_full_content = record_dieu8["content"]
            #print("Đã lấy nội dung Điều 8 từ KG.")
        else:
            print("LỖI: Không tìm thấy nội dung Điều 8 trong KG. Đảm bảo bạn đã chạy Cell 5 để tạo KG.")

        # Lấy nội dung Điều 5
        result_dieu5 = session.run("MATCH (a:Article {article_id: 'ARTICLE_LHNGD2014_DIEU5'}) RETURN a.content AS content")
        record_dieu5 = result_dieu5.single()
        if record_dieu5:
            dieu5_full_content = record_dieu5["content"]
            #print("Đã lấy nội dung Điều 5 từ KG.")
        else:
            print("LỖI: Không tìm thấy nội dung Điều 5 trong KG. Đảm bảo bạn đã chạy Cell 5 để tạo KG.")

        # Lấy nội dung các điểm cụ thể (ví dụ) 
        point_ids_to_fetch = [
            "POINT_LHNGD2014_DIEU5_KHOAN2_DIEMD",
            "POINT_LHNGD2014_DIEU8_KHOAN1_DIEMA"
        ]
        # và khoản 2 điều 8
        clause_id_to_fetch = "CLAUSE_LHNGD2014_DIEU8_KHOAN2"

        for point_id in point_ids_to_fetch:
            result_point = session.run("MATCH (p:Point {point_id: $pid}) RETURN p.content AS content", pid=point_id)
            record_point = result_point.single()
            if record_point:
                specific_points_data[point_id] = record_point["content"]
        
        result_clause = session.run("MATCH (c:Clause {clause_id: $cid}) RETURN c.content AS content", cid=clause_id_to_fetch)
        record_clause = result_clause.single()
        if record_clause:
            specific_points_data[clause_id_to_fetch] = record_clause["content"]


    if GROQ_API_KEY and dieu8_full_content != "N/A" and dieu5_full_content != "N/A":
        for i, user_q in enumerate(questions_to_test):
            print(f"\n--- Đang xử lý Câu hỏi {i+1}: {user_q} ---")

            current_specific_point_info = ""
            # Logic xác định specific_point_info dựa trên user_q
            if "ba đời" in user_q.lower() or "em họ" in user_q.lower():
                content = specific_points_data.get("POINT_LHNGD2014_DIEU5_KHOAN2_DIEMD", "")
                if content:
                    current_specific_point_info = f"Lưu ý cụ thể đến hành vi cấm tại Điểm d Khoản 2 Điều 5: '{content}'"
            elif "17 tuổi" in user_q or "18 tuổi" in user_q or "20 tuổi" in user_q:
                content = specific_points_data.get("POINT_LHNGD2014_DIEU8_KHOAN1_DIEMA", "")
                if content:
                    current_specific_point_info = f"Lưu ý cụ thể đến điều kiện độ tuổi tại Điểm a Khoản 1 Điều 8: '{content}'"
            elif "cùng giới" in user_q.lower():
                content = specific_points_data.get("CLAUSE_LHNGD2014_DIEU8_KHOAN2", "")
                if content:
                     current_specific_point_info = f"Lưu ý cụ thể đến quy định tại Khoản 2 Điều 8: '{content}'"


            chain_input = {
                "user_query": user_q,
                "dieu8_content": dieu8_full_content,
                "dieu5_content": dieu5_full_content,
                "specific_point_info": current_specific_point_info
            }
            # --- SỬA ĐỔI CÁCH LẤY KẾT QUẢ TỪ CHAIN ---
            llm_output_dict = law_qa_chain.invoke(chain_input)
            final_answer_text = llm_output_dict.get("text", "[Không có câu trả lời]")

            print(f"\nCâu hỏi: {user_q}") # In câu hỏi người dùng
            print(f"Trả lời:\n{final_answer_text}") # In câu trả lời đã được xử lý

    elif not GROQ_API_KEY:
        print("Không thể chạy demo hỏi đáp vì GROQ_API_KEY chưa được thiết lập.")
    else:
        print("Không thể truy xuất đầy đủ thông tin luật từ KG (Điều 5 hoặc Điều 8 bị thiếu). Hãy đảm bảo bạn đã chạy Cell 5 để tạo KG.")

# if driver:
#     driver.close()
#     print("\nĐã đóng kết nối Neo4j.")


--- Đang xử lý Câu hỏi 1: Chưa được xóa án tích có được đăng ký kết hôn không? ---

Câu hỏi: Chưa được xóa án tích có được đăng ký kết hôn không?
Trả lời:
Căn cứ vào Điều 8 Luật Hôn nhân và gia đình 2014 quy định về điều kiện kết hôn như sau:

1. Nam, nữ kết hôn với nhau phải tuân theo các điều kiện sau đây:
a) Nam từ đủ 20 tuổi trở lên, nữ từ đủ 18 tuổi trở lên;
b) Việc kết hôn do nam và nữ tự nguyện quyết định;
c) Không bị mất năng lực hành vi dân sự;
d) Việc kết hôn không thuộc một trong các trường hợp cấm kết hôn theo quy định tại các điểm a, b, c và d khoản 2 Điều 5 của Luật này.

Theo đó, để đăng ký kết hôn, người xin kết hôn phải không bị mất năng lực hành vi dân sự. Tuy nhiên, vấn đề người chưa được xóa án tích có được đăng ký kết hôn không được quy định rõ ràng trong Luật Hôn nhân và gia đình 2014.

Tuy nhiên, căn cứ vào Điều 5 Luật Hôn nhân và gia đình 2014 quy định về bảo vệ chế độ hôn nhân và gia đình như sau:

2. Cấm các hành vi sau đây:
d) Kết hôn hoặc chung sống như vợ 